<a href="https://colab.research.google.com/github/KelseyMarks/CS2316-Final/blob/main/Kelsey_and_Anna_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kelsey and Anna Phase II#
This is the data cleaning phase of the CS2316 Final Project.  For our project, we are looking at crime rates and how they compare to drug usage, education level, population density, and more in Atlanta, Boston, New York City, and Los Angeles.

The first thing we will do is import the needed tools for cleaning and gathering all of our data.

In [ ]:
import pandas as pd
import numpy as np
import csv
import json
from bs4 import BeautifulSoup
import requests

#Dataset 1 - The Downloaded Dataset#

For this dataset, we are cleaning a CSV file that contains the Atlanta Police Department Crime Log.  Cleaning this dataset will include removing unnecessary columns, adding an additional column for whether or not the crime was violent, and renaming the columns to be uniform with the other data sets we will be using.

In [ ]:
def data_parser():
  df = pd.read_csv("COBRA-2009-2019.csv")
  df = df[['Occur Date', 'Occur Time', 'UCR Literal']]
  df[['Year', 'Month', 'Day']] = df['Occur Date'].str.split('-', expand=True)
  df = df.drop(['Month', 'Day'], axis=1)
  df = df[df['Year'] == '2019']
  df = df.drop('Occur Date', axis=1)
  #print(df['UCR Literal'].value_counts())
  df['Violent'] = 'No'
  df.loc[df['UCR Literal']=='BURGLARY-RESIDENCE', 'Violent'] = 'Yes'
  df.loc[df['UCR Literal']=='AGG ASSAULT', 'Violent'] = 'Yes'
  df.loc[df['UCR Literal']=='BURGLARY-NONERES', 'Violent'] = 'Yes'
  df.loc[df['UCR Literal']=='HOMICIDE', 'Violent'] = 'Yes'
  df.loc[df['UCR Literal']=='MANSLAUGHTER', 'Violent'] = 'Yes'
  df.rename(columns={'Occur Time': 'Time', 'UCR Literal': 'Crime'}, inplace=True)
  print(df)
  df.to_csv('ATL Crime Cleaned.csv', index=False)

############ Function Call ############
data_parser()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.


        Time                 Crime  Year Violent
317884  0100  LARCENY-FROM VEHICLE  2019      No
317890  0020   LARCENY-NON VEHICLE  2019      No
317891  0120   LARCENY-NON VEHICLE  2019      No
317892  1740   LARCENY-NON VEHICLE  2019      No
317895  0400  LARCENY-FROM VEHICLE  2019      No
...      ...                   ...   ...     ...
342909  2030           AGG ASSAULT  2019     Yes
342910   432           AGG ASSAULT  2019     Yes
342911   920           AGG ASSAULT  2019     Yes
342912  1853           AGG ASSAULT  2019     Yes
342913  2045           AGG ASSAULT  2019     Yes

[24853 rows x 4 columns]


#Dataset 2 - Web Collection Requirement 1#

For this dataset, we wanted to get the personal income for each city of interest, New York City, Boston, Atlanta, and Los Angeles. We removed all other cities, as well as all other columns containing extra financial data about the cities.

In [ ]:
def web_parser1():
  final_list=[]
  final_dict={}
  response=requests.get("https://en.wikipedia.org/wiki/List_of_U.S._cities_by_adjusted_per_capita_personal_income")
  soup=BeautifulSoup(response.text,"html.parser")
  table_tag=soup.find("table")
  tr_tag=table_tag.find_all("tr")

  for tr in tr_tag:
    final_list.append(tr.text.strip())

  for city in final_list:
    city=city.split("\n")
    final_dict[city[0]]=city[1:]

  value_dict={}
  for key,value in final_dict.items():
    value_dict[key]=value[0]  

  value_dict["New York City"]=value_dict["1.  New York City-Newark-Jersey City, NY-NJ-PA MSA"]
  value_dict["Los Angeles"]=value_dict["2.  Los Angeles-Long Beach-Anaheim, CA MSA"]
  value_dict["Atlanta"]=value_dict["9.  Atlanta-Sandy Springs-Alpharetta, GA MSA"]
  value_dict["Boston"]=value_dict["11. Boston-Worcester-Providence, MA-RI-NH-CT CSA"]
  # print(value_dict)
  final_dict={}
  for key,value in value_dict.items():
    if key=="Boston" or key=="New York City" or key=="Atlanta" or key=="Los Angeles":
      final_dict[key]=value
  print(final_dict)
  f = open('CleanedIncome.txt', "w")
  f.write(str(final_dict))
  f.close()

############ Function Call ############
web_parser1()

{'New York City': '$79,844', 'Los Angeles': '$66,684', 'Atlanta': '$54,557', 'Boston': '$81,498'}


#Inconsistency 1#
In our first web collection requirement, we are looking at the average personal per capita income for the cities we are interested in. The main inconsistency looking at this data was that the city names were formatted differently than other datasets we are looking that. To fix this, we first used Beautiful Soup to get the html table from the website. We then went in and changed the dictionary of all the cities and their per capita incomes, to just include the ones we want. Finally, we changed the names to just be the city name itself to better match the formatting of the other datasets.

#Dataset 3 - Web Collection Requirement 2#

For this dataset, we accessed the FBI's API for their most wanted list in order to see the hometowns of those on the FBI's Most Wanted list.  To do this, we went through each page of the api and added the hometowns to a list.

In [ ]:
def web_parser2():
  hometowns = []
  hometowns_us = []
  hometowns_clean = []
  for n in range(1, 50):  # 50 pages
      response = requests.get(
          "https://api.fbi.gov/wanted/v1/list", params={"page": n}
      )
      data = json.loads(response.content)
      for i in range(1, len(data["items"])):
          if data["items"][i]["place_of_birth"] != None:
              hometowns.append(data["items"][i]["place_of_birth"])
  state_names = [
      "Alaska",
      "Alabama",
      "Arkansas",
      "American Samoa",
      "Arizona",
      "California",
      "Colorado",
      "Connecticut",
      "District ",
      "of Columbia",
      "Delaware",
      "Florida",
      "Georgia",
      "Guam",
      "Hawaii",
      "Iowa",
      "Idaho",
      "Illinois",
      "Indiana",
      "Kansas",
      "Kentucky",
      "Louisiana",
      "Massachusetts",
      "Maryland",
      "Maine",
      "Michigan",
      "Minnesota",
      "Missouri",
      "Mississippi",
      "Montana",
      "North Carolina",
      "North Dakota",
      "Nebraska",
      "New Hampshire",
      "New Jersey",
      "New Mexico",
      "Nevada",
      "New York",
      "Ohio",
      "Oklahoma",
      "Oregon",
      "Pennsylvania",
      "Puerto Rico",
      "Rhode Island",
      "South Carolina",
      "South Dakota",
      "Tennessee",
      "Texas",
      "Utah",
      "Virginia",
      "Virgin Islands",
      "Vermont",
      "Washington",
      "Wisconsin",
      "West Virginia",
      "Wyoming",
  ]
  for state in state_names:
      for place in hometowns:
          if state in place:
              hometowns_us.append(place)
  for place in hometowns_us:
      if len(place.split(",")) > 1:
          hometowns_clean.append(place.split(",")[1][1:])
      else:
          hometowns_clean.append(place.split(",")[0])
  hometowns_clean.remove("Gloucester County")
  hometowns_clean.remove("Washington State")
  hometowns_clean.remove("USA")
  hometowns_clean.remove("Queens")

  print(hometowns_clean)
  f = open('apiHometownsList.txt', "w")
  f.write(str(hometowns_clean))
  f.close()

############ Function Call ############
web_parser2()

['Alabama', 'Alabama', 'Arkansas', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'Colorado', 'Colorado', 'Connecticut', 'Connecticut', 'Delaware', 'Florida', 'Florida', 'Florida', 'Florida', 'Florida', 'Florida', 'Georgia', 'Georgia', 'Georgia', 'Hawaii', 'Idaho', 'Idaho', 'Illinois', 'Illinois', 'Illinois', 'Illinois', 'Illinois', 'United States of America', 'Illinois', 'Illinois', 'Illinois', 'Indiana', 'Indiana', 'Indiana', 'Kansas', 'Missouri', 'Missouri', 'Louisiana', 'Massachusetts', 'Massachusetts', 'Maryland', 'Michigan', 'Michigan', 'Michigan', 'Michigan', 'Michigan'

#Inconsistency 2#
In our API source for the FBI's most wanted list, we took out all unknown hometowns and removed all places that are not in the US. We identified these issues by creating a list of the hometowns then looking through it to see what needed to be changed to fit our project. They were significant because we are focusing our project on the US and we need no unknown values for our models to be made in phase 3. We took out the unknown values by using a for loop going through the list and used a similar method to get rid of the places not in the US. Also, some of the places only included a state or country, or were listed as state, country, or city, state, so we made the formatting uniform to be just the state by splitting strings and only taking the state names.  After all of this, there were still a couple of values that did not fit the standard syntax we wanted for our data of just the state names so we individually removed those values. 


#Additional Dataset 1#

For this data, we webscraped a table from the FBI website about crimes in the state of New York.  We were specifically looking for numbers based on the types of crimes that occurred in New York City so that is the specific data we webscraped for.

In [ ]:
def extra_source1():
  resp = requests.get('https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/tables/table-8/table-8-state-cuts/newyork.xls')
  soup = BeautifulSoup(resp.text, "html.parser")
  tags = soup.find_all('table')
  data = {}
  for tag in tags:
      place = tag.find_all('tr')
      for entry in place:
          name = entry.find('th').text.strip()
          if name == 'New York':
              violent = entry.find_all('td')[1].text.strip()
              murder = entry.find_all('td')[2].text.strip()
              rape = entry.find_all('td')[3].text.strip()
              robbery = entry.find_all('td')[4].text.strip()
              assault = entry.find_all('td')[5].text.strip()
              vandalism = entry.find_all('td')[6].text.strip()
              burglary = entry.find_all('td')[7].text.strip()
              larceny_theft = entry.find_all('td')[8].text.strip()
              vehicle_theft = entry.find_all('td')[9].text.strip()
              arson = entry.find_all('td')[10].text.strip()
              data = {'NYC':name, 'Violent':violent, 'Murder':murder, 'Rape':rape, 'Robbery':robbery,\
              'Assault':assault, 'Vandalism':vandalism, 'Burglary':burglary, 'Larceny/Theft': larceny_theft,\
              'Vehicle Theft':vehicle_theft, 'Arson':arson}
  print(data)
  with open("NYC Data Cleaned.txt", "w") as f:
      f.write(str(data))

############ Function Call ############
extra_source1()

{'NYC': 'New York', 'Violent': '47,821', 'Murder': '319', 'Rape': '2,770', 'Robbery': '13,396', 'Assault': '31,336', 'Vandalism': '122,299', 'Burglary': '9,846', 'Larceny/Theft': '106,931', 'Vehicle Theft': '5,522', 'Arson': ''}


#Additional Dataset 2#
For this data, we webscraped a table from WalletHub to get a score for the level of education as well as the availability of education for the needed cities for our project.  This website unfortunately blocks your IP if you try to webscrape more than twice so we will have to pick out only the cities we want for our project from the cleaned data in Phase III when we use the cleaned list of all the cities we have downloaded.

In [ ]:
def extra_source2():
  resp = requests.get('https://wallethub.com/edu/e/most-and-least-educated-cities/6656')
  soup = BeautifulSoup(resp.text, "html.parser")
#  soup = BeautifulSoup(open("WalletHub.html"), "html.parser")
  tags = soup.find_all('tbody')
  cities = {}
  for tag in tags:
      place = tag.find_all('tr')
      for entry in place:
          name = entry.find_all('td')[1].text
          total_score = entry.find_all('td')[2].text
          attain_edu = entry.find_all('td')[3].text
          quality = entry.find_all('td')[4].text
          cities[name] = [total_score, attain_edu, quality]
  print(cities)

######Commented out file writing in case this c
  f = open('EducationLevels.txt', "w")
  f.write(str(cities))
  f.close()

############ Function Call ############
extra_source2()

{'Ann Arbor, MI': ['93.99', '1', '1'], 'San Jose-Sunnyvale-Santa Clara, CA': ['82.03', '5', '5'], 'Washington-Arlington-Alexandria, DC-VA-MD-WV': ['81.78', '3', '27'], 'Madison, WI': ['80.83', '2', '49'], 'San Francisco-Oakland-Berkeley, CA': ['80.77', '4', '17'], 'Boston-Cambridge-Newton, MA-NH': ['78.26', '6', '47'], 'Durham-Chapel Hill, NC': ['78.06', '9', '4'], 'Raleigh-Cary, NC': ['77.08', '7', '44'], 'Seattle-Tacoma-Bellevue, WA': ['75.34', '10', '14'], 'Austin-Round Rock-Georgetown, TX': ['73.84', '11', '13'], 'Bridgeport-Stamford-Norwalk, CT': ['71.96', '8', '146'], 'Provo-Orem, UT': ['71.73', '17', '24'], 'Colorado Springs, CO': ['71.64', '14', '30'], 'Denver-Aurora-Lakewood, CO': ['70.78', '12', '79'], 'Trenton-Princeton, NJ': ['70.11', '15', '71'], 'Portland-South Portland, ME': ['69.58', '16', '70'], 'Portland-Vancouver-Hillsboro, OR-WA': ['69.24', '19', '50'], 'Tallahassee, FL': ['68.47', '22', '7'], 'Minneapolis-St. Paul-Bloomington, MN-WI': ['68.12', '13', '116'], 'Alban

#Additional Dataset 3#

There weren't any inconsistencies, however we did add in Atlanta manually from a Google search because the wiki page had been updated and Atlanta was removed.We used BeautifulSoup to webscrape the html page to get the table we wanted, since there were multiple on the webpage. We then went through and got the populations corresponding to the cities we wanted.


In [11]:
def extra_source3():
  final_list=[]
  final_dict={}
  response=requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population_density")
  soup=BeautifulSoup(response.text,"html.parser")
  # table_tag=soup.find("table")
  # h2_tag=soup.find("h2",{"class": "mw-headline"})
  table_tag=soup.find("table",{"class":"wikitable sortable"})
  tbody_tag=table_tag.find("tbody")
  # tr_tag=table_tag.find("tr")
  td_tag=tbody_tag.find_all("td")

  for td in td_tag:
    final_list.append(td.text.strip())

  for city in final_list:
    city=city.split("\n")
    final_dict[city[0]]=city[1:]
  # print(final_dict)
  new_dict={}
  for key,value in final_dict.items():
    if key=="8,175,133" or key=="645,149" or key=="337,977":
      new_dict[key]="placeholder"
  # print(new_dict)
  new_dict["8,175,133"]="New York City"
  new_dict["645,149"]="Boston"
  new_dict["337,977"]="Los Angeles"
  new_dict["429,414"]="Atlanta"
  print(new_dict)
  f = open('CleanedPop.txt', "w")
  f.write(str(final_dict))
  f.close()

############ Function Call ############
extra_source3()

{'8,175,133': 'New York City', '645,149': 'Boston', '337,977': 'Los Angeles', '429,414': 'Atlanta'}


#Additional Dataset 4#

For this dataset, we cleaned the CSV file for the crime log from the Los Angeles Police Department.  To do this, we removed the columns we didn't want, added the column to determine if a crime was violent, and standardized the column headers to match the other datasets.

In [ ]:
def extra_source4():
  df = pd.read_csv('Crime_Data_from_2010_to_2019.csv')
  df = df[['DATE OCC', 'TIME OCC', 'Crm Cd Desc', 'Weapon Desc']]
  df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])
  df['Year'] = pd.DatetimeIndex(df['DATE OCC']).year
  df = df.drop('DATE OCC', axis=1)
  df['Violent'] = 'Yes'
  df.loc[df['Weapon Desc'].isnull(), 'Violent'] = 'No'
  df = df.drop('Weapon Desc', axis=1)
  df = df[df['Year'] == 2019]
  df.rename(columns={'TIME OCC': 'Time', 'Crm Cd Desc': 'Crime'}, inplace=True)
  print(df)
  df.to_csv('LA Crime Cleaned.csv', index=False)

############ Function Call ############
extra_source4()

         Time                                            Crime  Year Violent
1392203  1500                                 VEHICLE - STOLEN  2019      No
1835837   130                                 VEHICLE - STOLEN  2019      No
1836035  1800                                 VEHICLE - STOLEN  2019      No
1836118   100                                    THEFT, PERSON  2019      No
1836286  1800                                    BIKE - STOLEN  2019      No
...       ...                                              ...   ...     ...
2119792   840          CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT  2019     Yes
2119793   400                                            ARSON  2019     Yes
2119794  2100           CRIMINAL THREATS - NO WEAPON DISPLAYED  2019     Yes
2119795  1800  THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)  2019      No
2119796  1615                            BURGLARY FROM VEHICLE  2019      No

[218088 rows x 4 columns]


#Inconsistency 3#

In our additional dataset for the LAPD Crime Logs, the DATE OCC column, which we are using to determine if the crime is in 2019 so we can be consistent with only using data from 2019, was a type object with format "02/20/2010 12:00:00AM".  This does not work for our formatting of trying to get only the year as an integer.  To fix this inconsistency, we changed the column to a datetime type and then used pd.DatetimeIndex(df['DATE OCC']).year to get just the year as an integer in its own column to best fit our needs of cleaning to the data so we only have data from 2019.  There is an additional inconsistency in this dataset with the Weapon Desc column.  A lot of crimes do not use weapons so instead the values are listed as NaN.  We can actually use this to our advantage when determining if the crime is violent or not.  The crime is nonviolent if the Weapon Desc is NaN and violent if a weapon is listed.

#Additional Dataset 5#

For this dataset, we did not really do any data cleaning but we wanted to include it to have the files ready for Phase III.  We manually took the the data from the images on the website and created a dictionary for it to output as a text file with the rest of the dictionaries for our project.

In [ ]:
def extra_source5():
  drug_use = {"Atlanta":{'Weed':60.5, 'Cocaine':15.3, 'Heroin':2.5, 'Meth':5.4},\
                "New York City":{'Weed':57.8, 'Cocaine':17.9, 'Heroin':2.0, 'Meth':9.8},\
                'Boston':{'Weed':61.3, 'Cocaine':18.6, 'Heroin':2.5, 'Meth':10.4},\
                'LA':{'Weed':60.1, 'Cocaine':20.7, 'Heroin':1.6, 'Meth':13.3}}
  print(drug_use)
  f = open('Drug Use Data.txt', "w")
  f.write(str(drug_use))
  f.close()

############ Function Call ############
extra_source5()

{'Atlanta': {'Weed': 60.5, 'Cocaine': 15.3, 'Heroin': 2.5, 'Meth': 5.4}, 'New York City': {'Weed': 57.8, 'Cocaine': 17.9, 'Heroin': 2.0, 'Meth': 9.8}, 'Boston': {'Weed': 61.3, 'Cocaine': 18.6, 'Heroin': 2.5, 'Meth': 10.4}, 'LA': {'Weed': 60.1, 'Cocaine': 20.7, 'Heroin': 1.6, 'Meth': 13.3}}


#Additional Dataset 6#

For this dataset, we cleaned the CSV file for the crime log for the Boston Police Department.  To do this, we removed the columns we didn't want and then added the column for whether or not the crime was violent.  We renamed the columns to match the rest of our datasets.

In [ ]:
def extra_source6():
  df = pd.read_csv('tmp9mkqyv6b.csv')
  df = df[['OFFENSE_DESCRIPTION', 'OCCURRED_ON_DATE', 'YEAR']]
  df[['Date', 'Time']] = df['OCCURRED_ON_DATE'].str.split(' ', expand=True)
  df = df.drop(['Date', 'OCCURRED_ON_DATE'], axis=1)
  df = df[df['YEAR'] == 2019]
  df[['Hour','Minute','Sec']] = df['Time'].str.split(':', expand=True)
  df['Time'] = df['Hour'] + df['Minute']
  df = df.drop(['Hour','Minute','Sec'], axis=1)
  df['Violent'] = 'No'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'M/V - LEAVING SCENE - PROPERTY DAMAGE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'ASSAULT SIMPLE - BATTERY', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'ASSAULT - SIMPLE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'ASSAULT AGGRAVATED', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'ASSAULT - AGGRAVATED - BATTERY', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'BURGLARY - RESIDENTIAL - FORCE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'M/V - LEAVING SCENE - PERSONAL INJURY', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'ASSAULT SIMPLE - BATTERYM/V ACCIDENT INVOLVING PEDESTRIAN - INJURY', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'BALLISTICS EVIDENCE/FOUND', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'WEAPON - FIREARM - CARRYING / POSSESSING, ETC', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'M/V ACCIDENT - INVOLVING PEDESTRIAN - INJURY', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'M/V ACCIDENT - INVOLVING BICYCLE - INJURY', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'BURGLARY - COMMERCIAL - FORCE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'WEAPON VIOLATION - CARRY/ POSSESSING/ SALE/ TRAFFICKING/ OTHER', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'DISTURBING THE PEACE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'M/V ACCIDENT - INVOLVING BICYCLE - INJURY', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'DISTURBING THE PEACE/ DISORDERLY CONDUCT/ GATHERING CAUSING ANNOYANCE/ NOISY PAR', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'DEMONSTRATIONS/RIOT', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'BREAKING AND ENTERING (B&E) MOTOR VEHICLE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'Migrated Report - Assault/Assault & Battery'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'INTIMIDATING WITNESS', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'BURGLARY - OTHER - FORCE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'Migrated Report - Burglary/Breaking and Entering', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'MURDER, NON-NEGLIGIENT MANSLAUGHTER', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'BREAKING AND ENTERING (B&E) MOTOR VEHICLE (NO PROPERTY STOLEN)', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'ROBBERY - HOME INVASION', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'ANIMAL ABUSE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'INJURY BICYCLE NO M/V INVOLVED', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'Migrated Report - Death Investigation', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'ROBBERY - BANK', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'WEAPON - FIREARM - OTHER VIOLATION', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'WEAPON - OTHER - OTHER VIOLATION', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'ARSON', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'CRIMINAL HARASSMENT', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'Migrated Report - Weapons Violation', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'Migrated Report - Criminal Homicide', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'KIDNAPPING/CUSTODIAL KIDNAPPING/ ABDUCTION', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'EXPLOSIVES - POSSESSION OR USE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'HOME INVASION', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'Migrated Report - Affray/Disturbing the Peace/Disorderly Conduct', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'FIREARM/WEAPON - ACCIDENTAL INJURY / DEATH', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'BIOLOGICAL THREATS', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'MANSLAUGHTER - VEHICLE - NEGLIGENCE', 'Violent'] = 'Yes'
  df.loc[df['OFFENSE_DESCRIPTION'] == 'THREATS TO DO BODILY HARM', 'Violent'] = 'Yes'
  df = df[df['OFFENSE_DESCRIPTION'] != 'Yes']
  df.rename(columns={'OFFENSE_DESCRIPTION': 'Crime', 'YEAR':'Year'}, inplace=True)
  print(df)
  df.to_csv('Boston Crime Cleaned.csv', index=False)

############ Function Call ############
extra_source6()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.


                                                   Crime  Year  Time Violent
0                              THREATS TO DO BODILY HARM  2019  1200     Yes
1                                     INVESTIGATE PERSON  2019  1630      No
2                            VAL - VIOLATION OF AUTO LAW  2019  2100      No
3                               PROPERTY - LOST/ MISSING  2019  1330      No
4                                         VERBAL DISPUTE  2019  0750      No
...                                                  ...   ...   ...     ...
87178             Migrated Report - Investigate Property  2019  0120      No
87179  Migrated Report - Aggravated Assault/Aggravate...  2019  0130      No
87181                   Migrated Report - Other Part III  2019  0221      No
87182                   Migrated Report - Other Part III  2019  0427      No
87183             Migrated Report - Investigate Property  2019  0639      No

[87145 rows x 4 columns]


#Inconsistency 4#

Similar to LAPD Crime Logs, the Boston Crime Report CSV file had another inconsistent formatting for the date/time.  To fix this, we split the 'OCCURRED_ON_DATE' column into the Date and Time columns.  To get the time column in consistent formatting with our other CSV files of crime logs from other cities, we had to split the time column and then recombine the hours and minutes to get a 3 or 4 digit number that represented the time at which the crime occured.  We were able to use the Year column to get the data from 2019.